#This RAG framework uses Contriever Retriever Embedding model with Milvus VectorDB and Mistral Ollama through llmam-index for generation

In [ ]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q torch sentence-transformers
!pip install -q -U pymilvus
!pip install -q datasets
!pip install -q transformers
!pip install -q spacy
!pip install -q tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 

In [ ]:
import pandas as pd
csv_path = "/content/halubench_short.csv"
data = pd.read_csv(csv_path)
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in data.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")
# Extract 'contexts' and 'questions'
text_data = data['contexts'].tolist()
questions_data = data['question'].tolist()
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

def split_text_by_length(text, max_length=4096):
    """
    Split the input text into chunks of max_length, ensuring no chunk exceeds max_length.
    """
    split_texts = []
    for i in range(0, len(text), max_length):
        chunk = text[i:i + max_length]
        if len(chunk) <= max_length:
            split_texts.append(chunk)
    return split_texts

split_texts = []
for text in text_data:
    split_texts.extend(split_text_by_length(text, max_length=4096))
#Initialize a list to store the mapping between questions and their context chunks
mapped_questions = []

# Map each question to its corresponding context chunks
for idx, text in enumerate(text_data):
    chunks = split_text_by_length(text, max_length=4096)  # Split context into chunks
    for _ in chunks:  # For each chunk of context, repeat the question
        mapped_questions.append(questions_data[idx])  # Map the question to each chunk

# Now both mapped_questions and split_texts should have the same length
assert len(mapped_questions) == len(split_texts), "Mapped questions and split texts are not aligned."

# Display the split chunks
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i+1}:")
    print(chunk)
    print()

Chunk 1:
Hoping to rebound from the road loss to the Chargers, the Rams went home for Week 9, as they fought the Kansas City Chiefs in a "Show Me State Showdown". The Chiefs struck first as RB Larry Johnson got a 1-yard TD run for the only score of the period.  In the second quarter, things got worse for the Rams as QB Damon Huard completed a 3-yard TD pass to TE Tony Gonzalez, while kicker Lawrence Tynes nailed a 42-yard field goal.  St. Louis got on the board with RB Steven Jackson getting a 2-yard TD run, yet Huard and Gonzalez hooked up with each other again on a 25-yard TD strike.  Rams kicker Jeff Wilkins made a 41-yard field goal to end the half.  In the third quarter, QB Marc Bulger completed a 2-yard TD pass to WR Kevin Curtis for the only score of the period, yet the only score of the fourth quarter came from Huard completing an 11-yard TD pass to TE Kris Wilson. With the loss, the Rams fell to 4-4.

Chunk 2:
As of the census of 2000, there were 218,590 people, 79,667 househo

#Retrieval

In [ ]:
from pymilvus import MilvusClient, utility

client = MilvusClient("milvus.db")
from pymilvus import (
    connections, FieldSchema, CollectionSchema, DataType, Collection, utility
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

# Connect to Milvus using the cluster information from Zilliz Cloud
connections.connect(
    alias="default",
    uri="https://in03-7a0bd7c7d07bbe2.serverless.gcp-us-west1.cloud.zilliz.com",
    token="2fd0d35c5daca9322a69814ef89023098b12df8a1e716334ac601149f386dbd4fd7b2ad5265df24b6a7452ca1ddd097c1fb572f8"
)

# Optionally, check the connection status
connected = connections.has_connection(alias="default")
print(f"Connection established: {connected}")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 9cd400b97d60486c8e23877f791ee6f7


Connection established: True


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
# Initialize tokenizer and model for Contriever retriever
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Function to mean pool token embeddings
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.0)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

In [ ]:
# Define collection schema
dense_dim = 768  # Dimension of embeddings
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=512),  # Field for question
    FieldSchema(name="context", dtype=DataType.VARCHAR, max_length=8192),  # Field for context
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),  # Dense vector field
]
schema = CollectionSchema(fields)

In [ ]:
col_name = "contriever_search"
if utility.has_collection(col_name):
    Collection(col_name).drop()  # Dropping existing collection (optional)
col = Collection(col_name, schema, consistency_level="Strong")

In [ ]:
# Initialize lists to store data for bulk insertion
questions_to_insert = []
contexts_to_insert = []
embeddings_to_insert = []

# Insert Data into Milvus
for idx, (question, context) in enumerate(zip(mapped_questions, split_texts)):
    # Tokenize and encode the context using Contriever
    ctx_input = tokenizer(context, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Generate token embeddings for the context
    with torch.no_grad():
        outputs = model(**ctx_input)

    # Apply mean pooling to get sentence embeddings
    ctx_emb = mean_pooling(outputs[0], ctx_input['attention_mask']).detach().numpy()

    # Flatten the numpy array to a list of floats
    ctx_emb = ctx_emb.flatten().tolist()

    # Append each field separately for insertion
    questions_to_insert.append(question)  # Append the question
    contexts_to_insert.append(context)    # Append the context
    embeddings_to_insert.append(ctx_emb)  # Append the dense vector

# Prepare the data in the correct format for Milvus insertion
data_to_insert = [questions_to_insert, contexts_to_insert, embeddings_to_insert]

In [ ]:
# Create Indices for Dense Vectors
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}  # Dense vector index
col.create_index("dense_vector", dense_index)

Status(code=0, message=)

In [ ]:
# Insert into Milvus collection
col.insert(data_to_insert)
# Flush the collection to ensure data is persisted
col.flush()

# Check the number of entities inserted
print(f"Number of entities inserted: {col.num_entities}")

Number of entities inserted: 92


In [ ]:
# Load Collection into Memory for Efficient Search
col.load()

#Generation

In [ ]:
!pip install llama-index-llms-ollama

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕▏  38 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  72 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 113 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 132 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   4% ▕▏ 162 MB/4.1 GB                  pulling manifest 
pulling ff8

In [ ]:
from llama_index.llms.ollama import Ollama
# Initialize the Ollama LLM
llm = Ollama(model="mistral", request_timeout=300.0, temperature=0.1)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import (Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate)

# Define the prompt template for generating the answer
prompt_template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    """

In [ ]:
questions_data = data['question'].tolist()
contexts_data = data['contexts'].tolist()

In [ ]:
# Function to mean pool token embeddings for sentence embeddings
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.0)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

# Generate embeddings using Contriever
def get_question_embedding(question):
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    question_emb = mean_pooling(outputs[0], inputs['attention_mask']).detach().numpy()
    return question_emb

In [ ]:
# Function to search Milvus for the best matching context based on question embedding
def dense_search(collection, embedding, limit=5):
    # Ensure the embedding is a list of floats and is wrapped in a list for Milvus search
    embedding = embedding.flatten().tolist() if isinstance(embedding, torch.Tensor) else embedding

    # Ensure each value is a float (Milvus requires this)
    embedding = [float(x) for x in embedding]  # Convert each element to a float
    embedding = [embedding]  # Milvus expects a list of embeddings (even if it's just one)

    # Perform a search in Milvus based on the dense embedding
    search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
    search_results = collection.search(embedding, "dense_vector", param=search_params, limit=limit)

    # Extract results (question and context)
    retrieved = [(hit.entity.get("question"), hit.entity.get("context")) for hit in search_results[0]]

    return retrieved

In [ ]:
# Initialize lists to store results
generated_answers = []
retrieved_contexts = []

# Loop through each question and generate an answer based on retrieved context
for i, question in enumerate(questions_data):
    # Generate question embedding
    question_emb = get_question_embedding(question)

    # Ensure that question_emb is converted to a plain list and is numeric
    if isinstance(question_emb, torch.Tensor):  # Check if it's a tensor
        question_emb = question_emb.cpu().detach().numpy()  # Convert tensor to NumPy array
    question_emb = question_emb.flatten()  # Flatten the embedding (to a 1D array)
    question_emb = [float(x) for x in question_emb]  # Convert each element to a float

    # Perform search using the dense vector and retrieve the best matching context
    retrieved_results = dense_search(col, question_emb, limit=5)  # Perform the dense search

    # Access the best result (first hit)
    best_question, best_context = retrieved_results[0]

    # Generate the answer using the Mistral model with the retrieved context and question
    prompt = f"Question: {question}\nContext: {best_context}\nAnswer:"
    response = llm.complete(prompt)

    # Append the generated answer and retrieved context to the respective lists
    generated_answers.append(response)
    retrieved_contexts.append(best_context)

    # Print the generated answer for each question
    print(f"Generated Answer for question '{question}':\n{response}")
    print('-' * 50)

Generated Answer for question 'Which team scored the longest field goal kick of the game?':
 To answer this question, I would need specific details about the game in question, such as the name of the game, the league it belongs to, and the date of the match. Without that information, I cannot provide an accurate answer.
--------------------------------------------------
Generated Answer for question 'How many percent were not  Irish?':
 The question does not provide a specific context or data to determine the percentage of people who were not Irish. To answer this question accurately, we would need information about the population in question and the number of individuals who were not Irish within that group.
--------------------------------------------------
Generated Answer for question 'How many yards was the second longest field goal of the first half?':
 The question does not provide any specific data about a football game or its statistics. To answer this question, I would need t

In [ ]:
questions = data['question'].tolist()
contexts = data['contexts'].tolist()
ground_truths = data['ground_truth'].tolist()
# Combine the result into a DataFrame
output_df = pd.DataFrame({
    'question': data['question'],
    'contexts': data['contexts'],
    'answer': generated_answers,    # Generated answers for each question
    'ground_truth': data['ground_truth']   # The ground truth from the original dataset
})
data['answer'] = generated_answers
data['retrieved_documents'] = retrieved_contexts

In [ ]:
output_csv_path = 'Contriever_retrieved_contexts.csv'
data.to_csv(output_csv_path, index=False)